<a href="https://colab.research.google.com/github/cgenevier/CSCI5622-HW4/blob/part-b-estimation-using-lang-features/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Study 1: Designing explainable speech-based machine learning models of depression

To open this ipynb in Colab, click the "Open in Colab" button at the top of the ipynb in Github, or [follow this link](https://colab.research.google.com/github/cgenevier/CSCI5622-HW4/blob/main/main.ipynb).

Given that Colab doesn't automatically load any of the content (data or other functions) from the Github repo, running the code below will copy the repo into the workspace directory for use. To save this ipynb file back to Github, select **File > Save** (which should show the repo if you're signed in) or **File > Save a copy in Github** if it's in the menu.

Note that the content of the data files or any of the other file structures are not saved back to Github, so make sure that if you make changes to things there, that you put them in Github separately.

In [1]:
# Clone Github Repo into the temporary local environment so data can be accessed and manipulated
!git clone https://github.com/cgenevier/CSCI5622-HW4.git
%cd CSCI5622-HW4

Cloning into 'CSCI5622-HW4'...
remote: Enumerating objects: 445, done.
remote: Counting objects: 100% (445/445), done.
remote: Compressing objects: 100% (440/440), done.
remote: Total 445 (delta 36), reused 394 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (445/445), 5.20 MiB | 10.27 MiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/CSCI5622-HW4


In [2]:
# Import necessary libraries

# Helpers
import glob

# Pandas, seaborn, and numpy for data manipulation
import pandas as pd
pd.set_option("display.max_rows", None)
import statistics as stat
import seaborn as sns
import numpy as np
np.random.seed(42)

# Keras & TensorFlow for building the neural networks
import itertools, json, time
from itertools import count
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers, callbacks, backend as K
tf.random.set_seed(42)

# Feature extraction
!pip install vaderSentiment transformers torch
from sklearn.feature_extraction.text import TfidfVectorizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import logging, BertTokenizer, BertModel
import torch

# Matplotlib for graphing
import matplotlib.pyplot as plt

# Disable progress bars (necessary for it to show up correctly in Github)
logging.set_verbosity_error()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.9 MB/s eta 0:00:00


##### Formatting the data - Depression Labels

In [3]:
# Import Depression Labels
# Columns: Participant_ID, PHQ_Score
depression_labels = pd.read_csv("data/DepressionLabels.csv")

# Rename Participant_ID to ParticipantID to match accoustic files below & force trimmed string type
depression_labels = depression_labels.rename(columns={"Participant_ID": "ParticipantID"})
depression_labels["ParticipantID"] = depression_labels["ParticipantID"].astype(str).str.strip()

##### Formatting the data - Text Features

In [4]:
# Import Text Dataset (for text feature extraction)
# Note: When comparing the E-DAIC_Transcripts files to the corresponding E-DAIC Acoustics files,
# it looks like the transcripts sometimes only contain partial data from the accoustics text column -
# for example, 386_Transcript.csv - so it seems to make sense to concatenate Text data in the
# Acoustics file for completeness.
rows = []
for p in glob.glob("data/E-DAIC_Acoustics/*_utterance_agg.csv"):
    df = pd.read_csv(p)
    df["ParticipantID"] = df["ParticipantID"].astype(str).str.strip()
    full_text = " ".join(df["Text"].dropna().astype(str))
    full_text = " ".join(full_text.split())  # collapse whitespace
    rows.append({"ParticipantID": df["ParticipantID"].iloc[0], "FullText": full_text})

# Columns: ParticipantID, FullText
text_df = pd.DataFrame(rows)
# Merge with labels. Columns: ParticipantID, FullText, PHQ_Score
lang_df = depression_labels.merge(text_df, on="ParticipantID", how="inner")

# Inspect results
lang_df.head()

,ParticipantID,PHQ_Score,FullText
0,386,11,might have pulled something that I'm going to ...
1,387,2,when she's done she'll let you know alrighty t...
2,388,17,are you okay with yes doing all right from Pas...
3,389,14,and please are you okay sure I'm okay small to...
4,390,9,and now she's going to chat with you for a bit...


##### Formatting the data - Acoustic Features

In [5]:
# Import Accoustic Dataset (for part c, d)

# Helper function for mean, standard dev, & IQR (interquartile range)
def summarize_cols(num_df: pd.DataFrame) -> pd.DataFrame:
    # mean and std per column (NaN-safe)
    mean_s = num_df.mean(numeric_only=True)
    std_s  = num_df.std(numeric_only=True)
    # IQR = Q3 - Q1 per column
    q75 = num_df.quantile(0.75, numeric_only=True)
    q25 = num_df.quantile(0.25, numeric_only=True)
    iqr_s = q75 - q25

    # assemble into a tidy (feature, stat) table
    stats = pd.concat(
        {"mean": mean_s, "std": std_s, "iqr": iqr_s},
        axis=1
    )  # index=feature, columns=[mean,std,iqr]

    # flatten to one row with columns like feature__mean
    wide = stats.stack().to_frame().T
    wide.columns = [f"{feat}__{stat}" for feat, stat in wide.columns]
    return wide

# Each file in E-DAIC_Acoustics contains utterance-level acoustic features for one participant.
rows_with_conf = []
rows_no_conf = []
for p in glob.glob("data/E-DAIC_Acoustics/*_utterance_agg.csv"):
    df = pd.read_csv(p)
    df["ParticipantID"] = df["ParticipantID"].astype(str).str.strip()

    # Include Confidence (column 5) + all acoustic features (6+)
    numeric_with_conf = df.columns[5:]
    df[numeric_with_conf] = df[numeric_with_conf].apply(pd.to_numeric, errors="coerce")

    agg_with_conf = summarize_cols(df[numeric_with_conf])
    agg_with_conf.insert(0, "ParticipantID", df["ParticipantID"].iloc[0])
    rows_with_conf.append(agg_with_conf)

    # Excludes Confidence - only include acoustic features
    numeric_no_conf = df.columns[6:]
    df[numeric_no_conf] = df[numeric_no_conf].apply(pd.to_numeric, errors="coerce")

    agg_no_conf = summarize_cols(df[numeric_no_conf])
    agg_no_conf.insert(0, "ParticipantID", df["ParticipantID"].iloc[0])
    rows_no_conf.append(agg_no_conf)

# Combine into one DataFrame each
acoustic_features_with_conf = pd.concat(rows_with_conf, ignore_index=True)
acoustic_features_no_conf = pd.concat(rows_no_conf, ignore_index=True)

# Merge with labels to add PHQ_Score
acoustic_df_with_confidence = depression_labels.merge(acoustic_features_with_conf, on="ParticipantID", how="inner")
acoustic_df_no_confidence   = depression_labels.merge(acoustic_features_no_conf, on="ParticipantID", how="inner")

# Reorder columns: ParticipantID, PHQ_Score, then features
cols = ["ParticipantID", "PHQ_Score"] + [c for c in acoustic_df_with_confidence.columns if c not in ["ParticipantID", "PHQ_Score"]]
acoustic_df_with_confidence = acoustic_df_with_confidence[cols]
cols = ["ParticipantID", "PHQ_Score"] + [c for c in acoustic_df_no_confidence.columns if c not in ["ParticipantID", "PHQ_Score"]]
acoustic_df_no_confidence = acoustic_df_no_confidence[cols]

# Inspect results
display(acoustic_df_with_confidence.head())
display(acoustic_df_no_confidence.head())

,ParticipantID,PHQ_Score,Confidence__mean,Confidence__std,Confidence__iqr,Loudness_sma3__mean,Loudness_sma3__std,Loudness_sma3__iqr,alphaRatio_sma3__mean,alphaRatio_sma3__std,...,F2frequency_sma3nz__iqr,F2amplitudeLogRelF0_sma3nz__mean,F2amplitudeLogRelF0_sma3nz__std,F2amplitudeLogRelF0_sma3nz__iqr,F3frequency_sma3nz__mean,F3frequency_sma3nz__std,F3frequency_sma3nz__iqr,F3amplitudeLogRelF0_sma3nz__mean,F3amplitudeLogRelF0_sma3nz__std,F3amplitudeLogRelF0_sma3nz__iqr
0,386,11,0.933917,0.055195,0.037800,0.147390,0.064438,0.089581,-15.719342,2.713920,...,42.335639,-128.087978,39.510693,63.620488,2584.433064,57.013353,47.965921,-131.009492,37.808278,59.861069
1,387,2,0.931755,0.062020,0.064689,0.094281,0.061988,0.060425,-17.290294,2.972634,...,117.523721,-140.359126,45.511084,86.525518,2525.715990,84.201997,76.811123,-142.535582,43.594940,82.822549
2,388,17,0.888103,0.105329,0.123520,0.103571,0.045205,0.069353,-16.480069,2.705549,...,81.395833,-156.991100,32.353199,56.016943,2549.702244,75.558349,94.163420,-158.777582,30.838568,53.547303
3,389,14,0.897373,0.099174,0.103688,0.095695,0.038370,0.055492,-20.799213,2.820568,...,81.503827,-143.443426,33.056194,47.504207,2549.110120,70.157241,97.720753,-145.861499,31.530769,45.670156
4,390,9,0.900761,0.082217,0.088130,0.106446,0.045064,0.052923,-17.288813,2.328784,...,79.791693,-139.494192,31.536717,39.076320,2450.182795,73.798789,106.373400,-142.697953,29.870122,36.615301


,ParticipantID,PHQ_Score,Loudness_sma3__mean,Loudness_sma3__std,Loudness_sma3__iqr,alphaRatio_sma3__mean,alphaRatio_sma3__std,alphaRatio_sma3__iqr,hammarbergIndex_sma3__mean,hammarbergIndex_sma3__std,...,F2frequency_sma3nz__iqr,F2amplitudeLogRelF0_sma3nz__mean,F2amplitudeLogRelF0_sma3nz__std,F2amplitudeLogRelF0_sma3nz__iqr,F3frequency_sma3nz__mean,F3frequency_sma3nz__std,F3frequency_sma3nz__iqr,F3amplitudeLogRelF0_sma3nz__mean,F3amplitudeLogRelF0_sma3nz__std,F3amplitudeLogRelF0_sma3nz__iqr
0,386,11,0.147390,0.064438,0.089581,-15.719342,2.713920,3.345443,26.607891,2.620843,...,42.335639,-128.087978,39.510693,63.620488,2584.433064,57.013353,47.965921,-131.009492,37.808278,59.861069
1,387,2,0.094281,0.061988,0.060425,-17.290294,2.972634,3.860120,27.380149,3.369106,...,117.523721,-140.359126,45.511084,86.525518,2525.715990,84.201997,76.811123,-142.535582,43.594940,82.822549
2,388,17,0.103571,0.045205,0.069353,-16.480069,2.705549,3.389694,26.983015,3.081380,...,81.395833,-156.991100,32.353199,56.016943,2549.702244,75.558349,94.163420,-158.777582,30.838568,53.547303
3,389,14,0.095695,0.038370,0.055492,-20.799213,2.820568,3.971909,31.428066,3.209966,...,81.503827,-143.443426,33.056194,47.504207,2549.110120,70.157241,97.720753,-145.861499,31.530769,45.670156
4,390,9,0.106446,0.045064,0.052923,-17.288813,2.328784,2.859256,29.355163,3.087194,...,79.791693,-139.494192,31.536717,39.076320,2450.182795,73.798789,106.373400,-142.697953,29.870122,36.615301


### (a) (2 points) Extracting language features.

**Syntactic vectorizers:** count vectorizer (e.g., CountVectorizer from sklearn) transforming
a collection of text documents into a numerical matrix of word or token counts; TFIDF vectorizer (e.g., TfidfVectorizer from sklearn) incorporating document-level weighting,
which emphasizes words significant to specific documents’ part-of-speech features counting
the distribution of part of speech tags over a document

In [6]:
# Use TfidfVectorizer from sklearn
vect = TfidfVectorizer(max_features=1000)
X_tfidf = vect.fit_transform(lang_df["FullText"])

# Convert sparse matrix to DataFrame
syntactic_df = pd.DataFrame(
    X_tfidf.toarray(),
    columns=vect.get_feature_names_out()
)

# Add ParticipantID column & move to first column
syntactic_df["ParticipantID"] = lang_df["ParticipantID"].values
cols = ["ParticipantID"] + [c for c in syntactic_df.columns if c != "ParticipantID"]
syntactic_df = syntactic_df[cols]

# Add back in PHQ_Score & move to second column
syntactic_df = syntactic_df.merge(depression_labels, on="ParticipantID", how="inner")
cols = ["ParticipantID", "PHQ_Score"] + [c for c in syntactic_df.columns if c not in ["ParticipantID", "PHQ_Score"]]
syntactic_df = syntactic_df[cols]

# Inspect dataframe
syntactic_df.head()

,ParticipantID,PHQ_Score,10,12,15,16,18,19,20,30,...,years,yes,yesterday,yet,york,you,young,younger,your,yourself
0,386,11,0.008630,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.017306,0.010627,0.00000,0.000000,0.000000,0.352617,0.011237,0.0,0.016921,0.019330
1,387,2,0.027755,0.000000,0.0,0.0,0.0,0.0,0.032038,0.0,...,0.074209,0.000000,0.00000,0.000000,0.047486,0.346512,0.072275,0.0,0.108838,0.031083
2,388,17,0.031186,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.041692,0.038404,0.00000,0.040133,0.000000,0.283166,0.000000,0.0,0.020382,0.034926
3,389,14,0.054573,0.052964,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.036479,0.000000,0.00000,0.000000,0.000000,0.325182,0.000000,0.0,0.089168,0.000000
4,390,9,0.000000,0.000000,0.0,0.0,0.0,0.0,0.016767,0.0,...,0.067967,0.008944,0.02011,0.000000,0.000000,0.123648,0.000000,0.0,0.009494,0.016268


**Semantic features:** sentiment scores (e.g., Vader, https://github.com/cjhutto/vaderSentiment),
topic distribution (using topic modeling), or named entities

In [7]:
# Using Vader to analyze sentiment of the text data
analyzer = SentimentIntensityAnalyzer()

# Apply Vader to the text data (creates 4 new columns)
vader_scores = lang_df["FullText"].apply(lambda x: pd.Series(analyzer.polarity_scores(str(x))))
semantic_df = pd.concat([lang_df, vader_scores], axis=1)

# Inspect dataframe
semantic_df.head()


,ParticipantID,PHQ_Score,FullText,neg,neu,pos,compound
0,386,11,might have pulled something that I'm going to ...,0.046,0.770,0.184,0.9999
1,387,2,when she's done she'll let you know alrighty t...,0.050,0.665,0.285,0.9996
2,388,17,are you okay with yes doing all right from Pas...,0.070,0.769,0.161,0.9953
3,389,14,and please are you okay sure I'm okay small to...,0.057,0.827,0.116,0.9822
4,390,9,and now she's going to chat with you for a bit...,0.067,0.740,0.193,0.9996


**Advanced features:** word embeddings, such as Word2Vec or BERT (e.g., pytorch-pretrainedbert)) for capturing contextual meaning

In [8]:
# Use BERT to capture contextual meaning (note: takes about 4 minutes to run on T4)

# Load uncased base model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
model.eval()

# Loop through text data and get embeddings
embeddings = []
for text in lang_df["FullText"]:
    # Truncate long text (BERT max = 512 tokens)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()  # [CLS] token
    embeddings.append(cls_embedding)

# Convert list of embeddings (each 768-dim) to DataFrame
bert_df = pd.DataFrame(np.vstack(embeddings))
bert_df.columns = [f"bert_{i}" for i in range(bert_df.shape[1])]

# Add ParticipantID and PHQ_Score
bert_df = pd.concat([lang_df[["ParticipantID", "PHQ_Score"]].reset_index(drop=True), bert_df], axis=1)

# Inspect dataframe
bert_df.head()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

,ParticipantID,PHQ_Score,bert_0,bert_1,bert_2,bert_3,bert_4,bert_5,bert_6,bert_7,...,bert_758,bert_759,bert_760,bert_761,bert_762,bert_763,bert_764,bert_765,bert_766,bert_767
0,386,11,0.024454,0.250100,0.480640,-0.209236,-0.064229,-0.409601,0.119546,0.926132,...,0.003123,-0.820313,-0.263505,-0.272018,0.424426,0.617825,-0.222068,-0.407773,0.242542,-0.204929
1,387,2,-0.085553,-0.097103,0.228180,0.072629,-0.162794,-0.457379,-0.005893,0.718336,...,0.354260,-1.054274,-0.246188,-0.318970,0.135778,0.865152,-0.351661,-0.323796,0.236854,-0.311687
2,388,17,-0.077363,0.149378,0.376864,0.118527,-0.100991,-0.355887,0.216300,0.758372,...,0.062571,-0.927725,-0.235411,-0.189597,0.669312,0.589867,-0.421785,-0.244376,0.100614,-0.064973
3,389,14,-0.305623,0.097802,0.182486,0.152354,-0.197408,-0.229550,0.315509,0.775767,...,0.160983,-1.146030,-0.109302,-0.369336,0.432707,0.457361,-0.329305,-0.434080,0.119847,-0.388673
4,390,9,-0.075243,-0.034517,0.310657,0.051467,-0.144259,-0.145556,0.093524,0.791170,...,-0.124969,-0.980402,-0.147668,-0.188621,0.437722,0.501311,-0.376553,-0.422698,0.162246,-0.142559


**Combined dataset:** Combined the three dataframes above into one with all the features

In [9]:
# Merge all three on ParticipantID
text_feature_df = (
    syntactic_df
    .merge(semantic_df, on=["ParticipantID", "PHQ_Score"], how="outer")
    .merge(bert_df, on=["ParticipantID", "PHQ_Score"], how="outer")
)
text_feature_df.head()

# Optional: sort by ParticipantID for clarity
#merged_df = merged_df.sort_values("ParticipantID").reset_index(drop=True)

,ParticipantID,PHQ_Score,10,12,15,16,18,19,20,30,...,bert_758,bert_759,bert_760,bert_761,bert_762,bert_763,bert_764,bert_765,bert_766,bert_767
0,386,11,0.008630,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.003123,-0.820313,-0.263505,-0.272018,0.424426,0.617825,-0.222068,-0.407773,0.242542,-0.204929
1,387,2,0.027755,0.000000,0.0,0.0,0.0,0.0,0.032038,0.0,...,0.354260,-1.054274,-0.246188,-0.318970,0.135778,0.865152,-0.351661,-0.323796,0.236854,-0.311687
2,388,17,0.031186,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.062571,-0.927725,-0.235411,-0.189597,0.669312,0.589867,-0.421785,-0.244376,0.100614,-0.064973
3,389,14,0.054573,0.052964,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.160983,-1.146030,-0.109302,-0.369336,0.432707,0.457361,-0.329305,-0.434080,0.119847,-0.388673
4,390,9,0.000000,0.000000,0.0,0.0,0.0,0.0,0.016767,0.0,...,-0.124969,-0.980402,-0.147668,-0.188621,0.437722,0.501311,-0.376553,-0.422698,0.162246,-0.142559


### (b) (2 points) Estimating depression severity with interpretable models using language features.

Question: In semantic feature extraction above, there are four features generated: neg, neu, pos, compound. They are inter-related because neg is the proportion of the document that is negative, neu is the proportion of the document that is neutral, pos is the proportion of the document that is positive, and compound is a normalized sentiment value that takes into account all 3. Should we remove some of these features because they're redundant?

### (c) (2 points) Estimating depression severity with interpretable models using acoustic features.

### (d) (2 points) Estimating depression severity with unimodal and multimodal deep learning models.

Question: In semantic feature extraction above, there are four features generated: neg, neu, pos, compound. They are inter-related because neg is the proportion of the document that is negative, neu is the proportion of the document that is neutral, pos is the proportion of the document that is positive, and compound is a normalized sentiment value that takes into account all 3. Should we remove some of these features because they're redundant?

### (e) (2 points) Explainable ML.

### (f) (Bonus, 2 points) Experimenting with transformers.